In [1]:
import os
import random
import math
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
import pandas as pd

In [2]:
base_dir = '/home/ubuntu/kaggle/flowers/xray_images/Data/'
train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')

In [3]:
"""
HyperParameters:
- Learning Rate 
- Momentum 
- Number of Conv Blocks
- Number of Layers in Each Block 
- Parameters of each Layer (Conv, Pooling, Flatten, Dense)
- Regularization 
- Batch Normalization 

"""

'\nHyperParameters:\n- Learning Rate \n- Momentum \n- Number of Conv Blocks\n- Number of Layers in Each Block \n- Parameters of each Layer (Conv, Pooling, Flatten, Dense)\n- Regularization \n- Batch Normalization \n\n'

In [4]:
def Conv2D_Input(filters, kernel_size, strides, padding, activation, kernel_regularizer, name, input_shape):
    return tf.keras.layers.Conv2D(filters = filters, kernel_size = kernel_size, 
                                  strides = strides, padding = padding, activation = activation, 
                                  kernel_regularizer = kernel_regularizer, name = name, 
                                  input_shape = input_shape)


def Conv2D(filters, kernel_size, strides, padding, activation, kernel_regularizer, name):
    return tf.keras.layers.Conv2D(filters = filters, kernel_size = kernel_size, 
                                  strides = strides, padding = padding, activation = activation, 
                                  kernel_regularizer = kernel_regularizer, name = name)

def MaxPool(pool_size, strides, padding, name):
    return tf.keras.layers.MaxPooling2D(pool_size = pool_size, strides = strides, 
                                        padding = padding, name = name)

def Flatten(name):
    return tf.keras.layers.Flatten(name = name)

def Dense(units, activation, kernel_regularizer, name):
    return tf.keras.layers.Dense(units = units, activation = activation, 
                                 kernel_regularizer=kernel_regularizer,name = name)

def Batch_Normalize():
    return tf.keras.layers.BatchNormalization(axis=1)

def DropOut(dropout_rate):
    return tf.keras.layers.Dropout(dropout_rate)

In [5]:
# Note - To be convereted to functions before posting
def set_hyperparameters(param):
    
    LR_10P_Range = param['LR_10P_Range'] 
    Mtm_Val = param['Mtm_Val'] 
    BS_Val = param['BS_Val']  
    Cv_Bks_Val= param['Cv_Bks_Val']  
    Cv_CvLy_Val= param['Cv_CvLy_Val']  
    Cv_PoLy_Val = param['Cv_PoLy_Val'] 
    Cv_KS_Val = param['Cv_KS_Val']  
    Cv_Reg_Val = param['Cv_Reg_Val']
    Cv_Fil_Val = param['Cv_Fil_Val']  
    Cv_Pd_Val = param['Cv_Pd_Val']  
    Cv_Sd_Val = param['Cv_Sd_Val']  
    Cv_Av_Val = param['Cv_Av_Val']  
    Po_PS_Val = param['Po_PS_Val'] 
    Po_Pd_Val = param['Po_Pd_Val'] 
    Dl_Num_Val = param['Dl_Num_Val']  
    Dl_Unt_Val = param['Dl_Unt_Val']  
    Dl_Reg_Val = param['Dl_Reg_Val']  
    Dl_Reg_Val = param['Dl_Reg_Val'] 
    Bch_Norm_Val = param['Bch_Norm_Val']  
    Dpout_Val = param['Dpout_Val']  
    Dp_Rate = param['Dp_Rate']
    Opt_Algo_Val= param['Opt_Algo_Val']  
    Input_Shape = param['Input_Shape']
    

    LR = pow(10,random.uniform(LR_10P_Range[0],LR_10P_Range[1]))
    
    random.shuffle(Mtm_Val)
    Mtm = Mtm_Val[0]
    
    random.shuffle(BS_Val)
    BS = BS_Val[0]
    
    random.shuffle(Cv_Bks_Val)
    Cv_Bks = Cv_Bks_Val[0]
    
    Cv_CvLy = []
    Cv_KS = []
    Cv_Reg = []
    Po_PS = []
    Po_Sd = []

    random.shuffle(Cv_KS_Val)
    random.shuffle(Cv_Reg_Val)
    random.shuffle(Po_PS_Val)
    random.shuffle(Cv_CvLy_Val)
    
    for i in range(Cv_Bks):
        Cv_CvLy.append(Cv_CvLy_Val[0])
        Cv_KS.append(Cv_KS_Val[0])
        Cv_Reg.append(Cv_Reg_Val[0])
        Po_PS.append(Po_PS_Val[0])
        Po_Sd.append(Po_PS_Val[0])

    Cv_Fil = []    
    random.shuffle(Cv_Fil_Val)

    for i in range(Cv_Bks):
        Cv_Fil.append(Cv_Fil_Val[0]*pow(2,math.ceil((i+1)/2)-1))  
        
    Cv_Pd = []
    Cv_Sd = []
    Cv_Av = []
    Cv_PoLy = []
    Po_Pd = []

    for i in range(Cv_Bks):
        Cv_Pd.append(Cv_Pd_Val)
        Cv_Sd.append(Cv_Sd_Val)
        Cv_Av.append(Cv_Av_Val)
        Cv_PoLy.append(Cv_PoLy_Val)
        Po_Pd.append(Po_Pd_Val)
    
    random.shuffle(Dl_Num_Val)
    Dl_Num = Dl_Num_Val[0]
    
    Dl_Unt =[]

    for i in range(Dl_Num):
        random.shuffle(Dl_Unt_Val)
        Dl_Unt.append(Dl_Unt_Val[0])
        
    Dl_Reg = []
    random.shuffle(Dl_Reg_Val)

    for i in range(Dl_Num):
        Dl_Reg.append(Dl_Reg_Val[0])
    
    Dl_Av = []

    for i in range(Dl_Num):
        Dl_Av.append(Dl_Av_Val)

    random.shuffle(Bch_Norm_Val)
    Bch_Norm = Bch_Norm_Val[0]
    
    random.shuffle(Dpout_Val)
    Dpout = Dpout_Val[0]
    
    random.shuffle(Opt_Algo_Val)
    Opt_Algo = Opt_Algo_Val[0]
    
    hyperparams = {'LR':LR, 'Mtm':Mtm, 'BS':BS,
                    'Cv_Bks':Cv_Bks, 'Cv_CvLy':Cv_CvLy, 'Cv_PoLy':Cv_PoLy, 
                    'Cv_KS':Cv_KS, 'Cv_Reg':Cv_Reg, 'Cv_Fil':Cv_Fil, 
                    'Cv_Pd':Cv_Pd, 'Cv_Sd':Cv_Sd, 'Cv_Av':Cv_Av, 
                    'Po_PS':Po_PS, 'Po_Pd':Po_Pd, 'Po_Sd':Po_Sd,
                    'Dl_Num':Dl_Num,'Dl_Unt':Dl_Unt, 'Dl_Reg':Dl_Reg, 'Dl_Av':Dl_Av, 
                    'Bch_Norm':Bch_Norm, 'Dpout':Dpout, 'Dp_Rate':Dp_Rate,
                    'Opt_Algo':Opt_Algo, 'Input_Shape':Input_Shape}
    
    return hyperparams

In [6]:
def generate_model(hyperparams):
    
    Cv_Bks  = hyperparams['Cv_Bks']
    Cv_CvLy = hyperparams['Cv_CvLy']
    Cv_PoLy = hyperparams['Cv_PoLy']
    Cv_KS = hyperparams['Cv_KS']
    Cv_Reg = hyperparams['Cv_Reg']
    Cv_Fil = hyperparams['Cv_Fil']
    Cv_Pd = hyperparams['Cv_Pd']
    Cv_Sd = hyperparams['Cv_Sd']
    Cv_Av = hyperparams['Cv_Av']
    Po_PS = hyperparams['Po_PS']
    Po_Pd = hyperparams['Po_Pd']
    Po_Sd = hyperparams['Po_Sd']
    Dl_Num = hyperparams['Dl_Num']
    Dl_Unt = hyperparams['Dl_Unt']
    Dl_Reg = hyperparams['Dl_Reg']
    Dl_Av = hyperparams['Dl_Av']
    Bch_Norm = hyperparams['Bch_Norm']
    Dpout = hyperparams['Dpout']
    Dp_Rate = hyperparams['Dp_Rate']
    
    Model = tf.keras.Sequential()
    
    for i in range(Cv_Bks):

        Cv_Bk_Name = 'Blk' + str(i+1)

        for j in range(Cv_CvLy[i]):
            if (i==0 and j==0):
                
                Ly_Name = Cv_Bk_Name + '_Conv' + str(j+1)
                Ly = Conv2D_Input(Cv_Fil[i], Cv_KS[i],
                                  Cv_Sd[i], Cv_Pd[i],
                                  Cv_Av[i], Cv_Reg[i],                         
                                  Ly_Name, Input_Shape)
                Model.add(Ly)

                if Bch_Norm == 'conv' or Bch_Norm == 'all':
                    Ly = Batch_Normalize()
                    Model.add(Ly)

                if Dpout == 'conv' or Dpout == 'all':
                    Ly = DropOut(Dp_Rate)
                    Model.add(Ly)

            else:
     

                Ly_Name = Cv_Bk_Name + '_Conv' + str(j+1)
                Ly = Conv2D(Cv_Fil[i], Cv_KS[i],
                            Cv_Sd[i], Cv_Pd[i],
                            Cv_Av[i], Cv_Reg[i],                         
                            Ly_Name)
                Model.add(Ly)
            
                if Bch_Norm == 'conv' or Bch_Norm == 'all':
                    Ly = Batch_Normalize()
                    Model.add(Ly)

                if Dpout == 'conv' or Dpout == 'all':
                    Ly = DropOut(Dp_Rate)
                    Model.add(Ly)

        for k in range(Cv_PoLy[i]):

            Ly_Name = Cv_Bk_Name + '_Pool' + str(k+1)
            Ly = MaxPool(Po_PS[i], Po_Sd[i],
                         Po_Pd[i], Ly_Name)
            Model.add(Ly)
        
    Model.add(Flatten('Flatten'))

    for l in range(Dl_Num):
        
        Ly_Name = 'Dense' + str(l+1)

        Ly = Dense(Dl_Unt[l], Dl_Av[l], Dl_Reg[l], Ly_Name)
        Model.add(Ly)

        if Bch_Norm == 'dense' or Bch_Norm == 'all':
            Ly = Batch_Normalize()
            Model.add(Ly)

        if Dpout == 'dense' or Dpout == 'all':
            Ly = DropOut(Dp_Rate)
            Model.add(Ly)

    Model.add(Dense(1, 'sigmoid', None, 'Output'))
    
    return Model

In [7]:
def prepare_callbacks(es_patience):
    callback_early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=es_patience,
                                                               verbose=1, restore_best_weights=True)
    #callback_model= ModelCheckpoint(filepath='/home/ubuntu/kaggle/flowers/xray_images/Outputs/Best_Model', 
                                    #save_best_only=True, save_weights_only=True)
    callbacks = [callback_early_stopping] #, callback_model]
    return callbacks

In [8]:
def compile_and_run_model(Model, Opt_Algo, LR, Mtm, Loss, Metrics):
    
    if Opt_Algo == 'sgd':
        Optimizer = tf.keras.optimizers.SGD(LR, Mtm)
        Model.compile(optimizer = Optimizer, loss = Loss, metrics = Metrics)
    
    elif Opt_Algo == 'adam':
        Optimizer = tf.keras.optimizers.Adam(learning_rate = LR)
        Model.compile(optimizer = Optimizer, loss = Loss, metrics = Metrics)

In [9]:
def prepare_train_test_data(train_dir, test_dir, batch_size, input_size):
    
    train_datagen = ImageDataGenerator(rescale=1.0/255.)
    test_datagen = ImageDataGenerator(rescale=1.0/255.)
    
    train_gen = train_datagen.flow_from_directory(train_dir,batch_size=batch_size,color_mode= 'rgb',
                                                    class_mode='binary',target_size=input_size) 
    test_gen = test_datagen.flow_from_directory(test_dir,batch_size=batch_size,color_mode= 'rgb',
                                                    class_mode='binary',target_size=input_size)
    
    return train_gen, test_gen

In [10]:
def export_results(iter_number, history, hyperparams, File_Name1, File_Name2):
    
    file = open(File_Name1,"w")
    file.write(str(hyperparams))
    file.close()
    
    df = pd.DataFrame.from_dict(history.history.items())
    
    df0 = pd.DataFrame()
    for i in range(7):
        df1 = pd.DataFrame(df[1][i],columns = [df[0][i]])
        df0 = pd.concat([df0, df1], axis=1)
    df0.to_csv(File_Name2)
    
    print("Files for iteration number "+ str(iter_number) + " have been exported.")

In [11]:
Input_Shape = (224,224,3) 
Input_Shape_WO_Channel = (224,224) 
Loss = 'binary_crossentropy'
Metrics = ['accuracy','Precision','Recall']

In [12]:
L2 = tf.keras.regularizers.l2(0.005)
Dp_Rate = 0.5

In [32]:
LR_10P_Range = [-2.8,-2.8]

In [33]:
LR = pow(10,random.uniform(LR_10P_Range[0],LR_10P_Range[1]))

In [37]:
Filters = [8,16]
Units = [128,512]

for i in Filters:
    for j in Units:
        print(i)
        print(j)

8
128
8
512
16
128
16
512


In [35]:
LR_10P_Range = [-2.8,-2.8] # Learning Rate 
Mtm_Val = [0] # Momentum 
BS_Val = [64] # Batch Size

Cv_Bks_Val = [5] # Number of Conv Blocks
Cv_CvLy_Val = [1] # Num of Conv Layers by Conv Blocks (Can be different in each Conv Block)
Cv_PoLy_Val = 1 # Num of Pooling layers by Conv Blocks (Same by each Conv Block)

Cv_KS_Val = [3] # Kernerl Size in Conv Layers by Conv Blocks (Same across all Conv Blocks) 
Cv_Reg_Val = [None] # Regularizaion Type in Conv Layers by Conv Blocks (Same across all Conv Blocks)
Cv_Fil_Val = [8] # Number of Filters in Conv Layers by Conv Blocks (This fixes the first, rest are followed a-a-2a-2a-4a)
Cv_Pd_Val = 'same'
Cv_Sd_Val = (1,1)
Cv_Av_Val = 'relu'

Po_PS_Val = [(2,2)] # Pool Size in Pool Layers by Conv Blocks 
Po_Pd_Val = 'same'

Dl_Num_Val = [1]
Dl_Unt_Val = [128]
Dl_Reg_Val = [None]
Dl_Av_Val = 'relu'

Bch_Norm_Val = ['none']
Dpout_Val = ['none']
Opt_Algo_Val = ['sgd']

param_values = {'LR_10P_Range':LR_10P_Range, 'Mtm_Val':Mtm_Val, 'BS_Val':BS_Val,
                'Cv_Bks_Val':Cv_Bks_Val, 'Cv_CvLy_Val':Cv_CvLy_Val, 'Cv_PoLy_Val':Cv_PoLy_Val, 
                'Cv_KS_Val':Cv_KS_Val, 'Cv_Reg_Val':Cv_Reg_Val, 
                'Cv_Fil_Val':Cv_Fil_Val, 'Cv_Pd_Val':Cv_Pd_Val, 
                'Cv_Sd_Val':Cv_Sd_Val, 'Cv_Av_Val':Cv_Av_Val, 
                'Po_PS_Val':Po_PS_Val, 'Po_Pd_Val':Po_Pd_Val, 
                'Dl_Num_Val':Dl_Num_Val, 'Dl_Unt_Val':Dl_Unt_Val,
                'Dl_Reg_Val':Dl_Reg_Val, 'Dl_Av_Val':Dl_Av_Val, 
                'Bch_Norm_Val':Bch_Norm_Val, 'Dpout_Val':Dpout_Val,
                'Dp_Rate':Dp_Rate, 'Opt_Algo_Val':Opt_Algo_Val,
                'Input_Shape':Input_Shape}

In [31]:
os.chdir('/home/ubuntu/kaggle/flowers/xray_images/Output_Iter2')

In [ ]:
for i in range(1):
    
    hyperparams = set_hyperparameters(param_values)
    Model = generate_model(hyperparams)
    
    Model.summary()

    print("")
    print("Batch_Size: "+str(hyperparams['BS']))
    print("Learning_Rate: "+str(hyperparams['LR']))
    print("Momentum: "+str(hyperparams['Mtm']))
    print("Optimization_Algo: "+str(hyperparams['Opt_Algo']))
    print("Regularization_Cv_Layers" + str(hyperparams['Cv_Reg']))
    print("Regularization_Dense_Layers" + str(hyperparams['Dl_Reg']))
    print ()
    print("")

    train_gen, test_gen = prepare_train_test_data(train_dir, test_dir, 
                                                      hyperparams['BS'], Input_Shape_WO_Channel)
    compile_and_run_model(Model, hyperparams['Opt_Algo'], hyperparams['LR'], hyperparams['Mtm'],
                              Loss, Metrics)
    callbacks = prepare_callbacks(10)
    
    history = Model.fit(train_gen, validation_data= test_gen,
                            epochs = 30, verbose=1, callbacks = callbacks)
    
    File_Name1 = 'Hyperparams_Model_' + str(i) + '.txt'
    File_Name2 = 'Model_Results_' + str(i) + '.csv'
    
    export_results(i, history, hyperparams, File_Name1, File_Name2)
    del(Model, history, train_gen, test_gen, hyperparams)

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Blk1_Conv1 (Conv2D)          (None, 224, 224, 16)      448       
_________________________________________________________________
Blk1_Conv2 (Conv2D)          (None, 224, 224, 16)      2320      
_________________________________________________________________
Blk1_Pool1 (MaxPooling2D)    (None, 112, 112, 16)      0         
_________________________________________________________________
Blk2_Conv1 (Conv2D)          (None, 112, 112, 16)      2320      
_________________________________________________________________
Blk2_Conv2 (Conv2D)          (None, 112, 112, 16)      2320      
_________________________________________________________________
Blk2_Pool1 (MaxPooling2D)    (None, 56, 56, 16)        0         
_________________________________________________________________
Blk3_Conv1 (Conv2D)          (None, 56, 56, 32)       

Found 624 images belonging to 2 classes.
Epoch 1/30
41/41 [==============================] - 56s 1s/step - loss: 0.6870 - acc: 0.7379 - precision_22: 0.7422 - recall_22: 0.9917 - val_loss: 0.6870 - val_acc: 0.6250 - val_precision_22: 0.6250 - val_recall_22: 1.0000
Epoch 2/30
41/41 [==============================] - 53s 1s/step - loss: 0.6750 - acc: 0.7429 - precision_22: 0.7429 - recall_22: 1.0000 - val_loss: 0.6815 - val_acc: 0.6250 - val_precision_22: 0.6250 - val_recall_22: 1.0000
Epoch 3/30
41/41 [==============================] - 53s 1s/step - loss: 0.6631 - acc: 0.7429 - precision_22: 0.7429 - recall_22: 1.0000 - val_loss: 0.6764 - val_acc: 0.6250 - val_precision_22: 0.6250 - val_recall_22: 1.0000
Epoch 4/30
41/41 [==============================] - 53s 1s/step - loss: 0.6503 - acc: 0.7429 - precision_22: 0.7429 - recall_22: 1.0000 - val_loss: 0.6711 - val_acc: 0.6250 - val_precision_22: 0.6250 - val_recall_22: 1.0000
Epoch 5/30
41/41 [==============================] - 53s 1s/step

Found 624 images belonging to 2 classes.
Epoch 1/30
163/163 [==============================] - 53s 325ms/step - loss: 0.6770 - acc: 0.7320 - precision_28: 0.7434 - recall_28: 0.9763 - val_loss: 0.6781 - val_acc: 0.6250 - val_precision_28: 0.6250 - val_recall_28: 1.0000
Epoch 2/30
163/163 [==============================] - 52s 318ms/step - loss: 0.6477 - acc: 0.7429 - precision_28: 0.7429 - recall_28: 1.0000 - val_loss: 0.6668 - val_acc: 0.6250 - val_precision_28: 0.6250 - val_recall_28: 1.0000
Epoch 3/30
163/163 [==============================] - 52s 318ms/step - loss: 0.6172 - acc: 0.7429 - precision_28: 0.7429 - recall_28: 1.0000 - val_loss: 0.6612 - val_acc: 0.6250 - val_precision_28: 0.6250 - val_recall_28: 1.0000
Epoch 4/30
163/163 [==============================] - 52s 317ms/step - loss: 0.5893 - acc: 0.7429 - precision_28: 0.7429 - recall_28: 1.0000 - val_loss: 0.6690 - val_acc: 0.6250 - val_precision_28: 0.6250 - val_recall_28: 1.0000
Epoch 5/30
163/163 [=======================

Epoch 00008: early stopping
Files for iteration number 7 have been exported.
Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Blk1_Conv1 (Conv2D)          (None, 224, 224, 8)       224       
_________________________________________________________________
Blk1_Conv2 (Conv2D)          (None, 224, 224, 8)       584       
_________________________________________________________________
Blk1_Pool1 (MaxPooling2D)    (None, 112, 112, 8)       0         
_________________________________________________________________
Blk2_Conv1 (Conv2D)          (None, 112, 112, 8)       584       
_________________________________________________________________
Blk2_Conv2 (Conv2D)          (None, 112, 112, 8)       584       
_________________________________________________________________
Blk2_Pool1 (MaxPooling2D)    (None, 56, 56, 8)         0         
__________________________________________

Epoch 1/30
163/163 [==============================] - 55s 338ms/step - loss: 0.5858 - acc: 0.7391 - precision_34: 0.7425 - recall_34: 0.9933 - val_loss: 0.6804 - val_acc: 0.6250 - val_precision_34: 0.6250 - val_recall_34: 1.0000
Epoch 2/30
163/163 [==============================] - 52s 320ms/step - loss: 0.5736 - acc: 0.7429 - precision_34: 0.7429 - recall_34: 1.0000 - val_loss: 0.6764 - val_acc: 0.6250 - val_precision_34: 0.6250 - val_recall_34: 1.0000
Epoch 3/30
163/163 [==============================] - 52s 321ms/step - loss: 0.5715 - acc: 0.7429 - precision_34: 0.7429 - recall_34: 1.0000 - val_loss: 0.7181 - val_acc: 0.6250 - val_precision_34: 0.6250 - val_recall_34: 1.0000
Epoch 4/30
163/163 [==============================] - 52s 318ms/step - loss: 0.5713 - acc: 0.7429 - precision_34: 0.7429 - recall_34: 1.0000 - val_loss: 0.6887 - val_acc: 0.6250 - val_precision_34: 0.6250 - val_recall_34: 1.0000
Epoch 5/30
163/163 [==============================] - 52s 321ms/step - loss: 0.5716 

Found 624 images belonging to 2 classes.
Epoch 1/30
41/41 [==============================] - 56s 1s/step - loss: 0.6811 - acc: 0.7429 - precision_38: 0.7429 - recall_38: 1.0000 - val_loss: 0.6831 - val_acc: 0.6250 - val_precision_38: 0.6250 - val_recall_38: 1.0000
Epoch 2/30
41/41 [==============================] - 53s 1s/step - loss: 0.6607 - acc: 0.7429 - precision_38: 0.7429 - recall_38: 1.0000 - val_loss: 0.6741 - val_acc: 0.6250 - val_precision_38: 0.6250 - val_recall_38: 1.0000
Epoch 3/30
41/41 [==============================] - 53s 1s/step - loss: 0.6348 - acc: 0.7429 - precision_38: 0.7429 - recall_38: 1.0000 - val_loss: 0.6657 - val_acc: 0.6250 - val_precision_38: 0.6250 - val_recall_38: 1.0000
Epoch 4/30
41/41 [==============================] - 53s 1s/step - loss: 0.6017 - acc: 0.7429 - precision_38: 0.7429 - recall_38: 1.0000 - val_loss: 0.6716 - val_acc: 0.6250 - val_precision_38: 0.6250 - val_recall_38: 1.0000
Epoch 5/30
41/41 [==============================] - 53s 1s/step

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
Epoch 1/30
163/163 [==============================] - 54s 330ms/step - loss: 0.6283 - acc: 0.7345 - precision_42: 0.7424 - recall_42: 0.9840 - val_loss: 0.6853 - val_acc: 0.6250 - val_precision_42: 0.6250 - val_recall_42: 1.0000
Epoch 2/30
163/163 [==============================] - 51s 315ms/step - loss: 0.5743 - acc: 0.7429 - precision_42: 0.7429 - recall_42: 1.0000 - val_loss: 0.6918 - val_acc: 0.6250 - val_precision_42: 0.6250 - val_recall_42: 1.0000
Epoch 3/30
163/163 [==============================] - 52s 322ms/step - loss: 0.5703 - acc: 0.7429 - precision_42: 0.7429 - recall_42: 1.0000 - val_loss: 0.6903 - val_acc: 0.6250 - val_precision_42: 0.6250 - val_recall_42: 1.0000
Epoch 4/30
163/163 [==============================] - 52s 316ms/step - loss: 0.5668 - acc: 0.7429 - precision_42: 0.7429 - recall_42: 1.0000 - val_loss: 0.7004 - val_acc: 0.6250 - val_precision_42: 0.6250 - val_recall_42: 1.0000
E

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
Epoch 1/30
82/82 [==============================] - 57s 690ms/step - loss: 0.8799 - acc: 0.7429 - precision_44: 0.7429 - recall_44: 1.0000 - val_loss: 0.6894 - val_acc: 0.6250 - val_precision_44: 0.6250 - val_recall_44: 1.0000
Epoch 2/30
82/82 [==============================] - 52s 640ms/step - loss: 0.5739 - acc: 0.7429 - precision_44: 0.7429 - recall_44: 1.0000 - val_loss: 0.6778 - val_acc: 0.6250 - val_precision_44: 0.6250 - val_recall_44: 1.0000
Epoch 3/30
82/82 [==============================] - 53s 649ms/step - loss: 0.5736 - acc: 0.7429 - precision_44: 0.7429 - recall_44: 1.0000 - val_loss: 0.6897 - val_acc: 0.6250 - val_precision_44: 0.6250 - val_recall_44: 1.0000
Epoch 4/30
82/82 [==============================] - 53s 651ms/step - loss: 0.5707 - acc: 0.7429 - precision_44: 0.7429 - recall_44: 1.0000 - val_loss: 0.7057 - val_acc: 0.6250 - val_precision_44: 0.6250 - val_recall_44: 1.0000
Epoch 5/3